In [1]:
import sys
import logging
import os
import json
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(message)s")

# LOAD DATA

In [2]:
DATA_FOLDER = os.getcwd() + '/review_scorer/data/'
SENTI_PATH = DATA_FOLDER + 'SentiWord_info.json'
DATA_PATH = DATA_FOLDER + 'data_origin.csv'
data: pd.DataFrame

with open(SENTI_PATH, mode='rt', encoding='UTF8') as f:
    senti = pd.DataFrame.from_dict(json.load(f))

data = pd.read_csv(DATA_PATH, encoding='UTF8')
data = data.dropna(axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# TOKENIZER

In [3]:
from tqdm import tqdm
from twkorean import TwitterKoreanProcessor

processor = TwitterKoreanProcessor()
tokenize = processor.tokenize_to_strings
tokens = [tokenize(_) for _ in tqdm(data.review[:5000])]

100%|████████████████████████████████████| 71904/71904 [08:29<00:00, 141.08it/s]


# SET REVIEW SCORER

In [4]:
from review_scorer import Doc2Category

# Review scorer needs tokens of datas to train when initializing it.
# 리뷰 채점기 클래스를 생성할 때, 토크나이즈 된 데이터를 인자로 주어야 합니다.
rs = Doc2Category(sentences=tokens, senti_dict_path=SENTI_PATH)

collecting all words and their counts
PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
collected 20272 word types from a corpus of 545838 raw words and 5000 sentences
Creating a fresh vocabulary
Doc2Category lifecycle event {'msg': 'effective_min_count=5 retains 6079 unique words (29.99% of original 20272, drops 14193)', 'datetime': '2022-08-12T12:06:16.521589', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-65-generic-x86_64-with-debian-buster-sid', 'event': 'prepare_vocab'}
Doc2Category lifecycle event {'msg': 'effective_min_count=5 leaves 523286 word corpus (95.87% of original 545838, drops 22552)', 'datetime': '2022-08-12T12:06:16.522415', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-65-generic-x86_64-with-debian-buster-sid', 'event': 'prepare_vocab'}
deleting the raw counts diction

## TAGGING RIVIEW SCORER's SENTIMENTAL DICTIONARY

In [9]:
# Tagging review scorer's sentimental dictionary by category.
# 카테고리에 따라 리뷰 채점기의 감성사전을 태깅합니다.
rs.tag(categories={'taste': ['맛', '맛있다', '맛없다'],
                   'price': ['가격', '싸다', '비싸다', '저렴'],
                   'service': ['서비스', '친절'],
                   'atmosphere': ['인테리어', '분위기']},
       width=4, depth=4)

category "taste" tagged
category "price" tagged
category "service" tagged
category "atmosphere" tagged


# SCORING WITH REVIEW SCORER

In [10]:
start = 4000
for i in range(start, start + 10):
    print('index: ', i)
    print(rs.score_review(tokenize(data.review.iloc[i])))
    print(data.review.iloc[i])
    print()

index:  4000
{'taste': 0, 'price': 0, 'service': 0, 'atmosphere': 0}
괜찮네

index:  4001
{'taste': 0, 'price': 0, 'service': 0, 'atmosphere': 0}
The staff is very helpful the waiting area is very good and efficient the wait for a table was not long since the restaurant is a large space with many tables the tables are clean the staff is really good at what they do. The deboning of the fish was a show of their skills and the food is good . I would definitely come back here again in the future

index:  4002
{'taste': 2, 'price': 0, 'service': 2, 'atmosphere': 0}
역시 주말 합정 8시 쉽지 않아요 ㅋㅋ (지난 가을 방문) 웨이팅이 꽤 있어서 기다리는 동안 아예 건너펀 합정스케줄에서 와인으로 1차 끝나고 넘어갔습니다. 그런데 너무 맛있어서 누가 보면 여기가 1차구나 싶을정도로 먹어버렸네요 ㅋㅋ   세트로 시켰을때 채소꼬치가 하나도 안 나오는거 보면 채소싫어하는 초딩입맛도 걱정할 필요 없을 것 같아요 친구들이랑 한잔하면서 수다떠느라 정신 없어서 직원분이 하나하나 친절하게 설명해주신것 거의 다 까먹었어요...ㅎ 닭안심은 안심살답게 보들보들했어요! 보들보들한데 닭다리살보다는 담백해서 술안주로 딱 ㅎㅎ  친구는 아킬레스건이 참 맛있었다고...!  다들 술취해서 자기가 맛있게 먹은거 하나씩 기억하더라고요ㅋㅋㅋ 공통적으로 내린 결론은 쿠이신보 진짜 맛있었다는 거....ㅎ 가라아게도 강추입니다!'

index:  4003
{'taste': 6,

# PRINT TAGGED WORDS

In [11]:
rs.print_trees()

taste
├── 맛
│   ├── 느낌
│   │   ├── 맛
│   │   │   ├── 느낌
│   │   │   ├── 부분
│   │   │   ├── 감칠맛
│   │   │   └── 약간
│   │   ├── 스타일
│   │   │   ├── 취향
│   │   │   ├── 수준
│   │   │   ├── 타입
│   │   │   └── 특징
│   │   ├── 약간
│   │   │   ├── 살짝
│   │   │   ├── 육향
│   │   │   ├── 느껴지다
│   │   │   └── 부분
│   │   └── 부분
│   │       ├── 감
│   │       ├── 짠맛
│   │       ├── 약간
│   │       └── 단맛
│   ├── 부분
│   │   ├── 감
│   │   │   ├── 부분
│   │   │   ├── 전혀
│   │   │   ├── 질
│   │   │   └── 거부
│   │   ├── 짠맛
│   │   │   ├── 신맛
│   │   │   ├── 불맛
│   │   │   ├── 단맛
│   │   │   └── 육향
│   │   ├── 약간
│   │   │   ├── 살짝
│   │   │   ├── 육향
│   │   │   ├── 느껴지다
│   │   │   └── 부분
│   │   └── 단맛
│   │       ├── 감칠맛
│   │       ├── 육향
│   │       ├── 향
│   │       └── 지방
│   ├── 감칠맛
│   │   ├── 단맛
│   │   │   ├── 감칠맛
│   │   │   ├── 육향
│   │   │   ├── 향
│   │   │   └── 지방
│   │   ├── 식감
│   │   │   ├── 감칠맛
│   │   │   ├── 단맛
│   │   │   ├── 향
│   │   │   └── 속
│   │   ├── 향
│   │   │   ├── 감칠맛
│   │   │

In [13]:
for root in rs.word_roots:
    words_set = list(map(lambda x: x.data, root.descendants))
    print(f'{root.name}({len(words_set)}개)')
    print(*words_set)

taste(255개)
맛 느낌 맛 느낌 부분 감칠맛 약간 스타일 취향 수준 타입 특징 약간 살짝 육향 느껴지다 부분 부분 감 짠맛 약간 단맛 부분 감 부분 전혀 질 거부 짠맛 신맛 불맛 단맛 육향 약간 살짝 육향 느껴지다 부분 단맛 감칠맛 육향 향 지방 감칠맛 단맛 감칠맛 육향 향 지방 식감 감칠맛 단맛 향 속 향 감칠맛 단맛 식감 육향 육향 단맛 감칠맛 짠맛 은은하다 약간 살짝 약간 간이 씹다 기름 육향 단맛 감칠맛 짠맛 은은하다 느껴지다 강하다 덜 약간 깊다 부분 감 짠맛 약간 단맛 맛있다 잘 자다 잘 고수 어울리다 향신료 감사하다 술술 도저히 쭉쭉 찢어지다 터지다 팡팡 고기랑 새우다 살이 지다 술술 신기하다 일수 은근 배부르다 이렇게 .... 뭐 근데 ㅠ 남기다 불다 .... 눈치 빨리 불다 빨리 남기다 고프다 눈치 맛없다 그나마 빼다 그런데 그러나 완전 대박 짱 존맛 완전 혜자 짱맛 !!!!! 보다도 핵 ,,, 물김치 죽이다 그럭저럭 동치미 난도 ,,, 그럭저럭 난도 ...... 진짜진짜 만족스럽다 다만 하지만 그러나 나름 매우 즐겁다 너무나 재밌다 아무리 애정 너무나 전망 잘되다 좋아지다 자꾸만 예쁘다 전망 고급스럽다 할머니 잘되다 맛없다 그나마 그걸 그나마 아무튼 봣 대요 이상하다 뭐라다 색다르다 그저 그나마 도대체 ;; ㅋ 겠다 패스 ㅋ 도대체 암튼 몇개 영 빼다 이상하다 뭐라다 색다르다 그저 그나마 색다르다 이상하다 뭐라다 재밌다 그저 부담스럽다 그만큼 심하다 엄청나다 낮다 맛없다 그나마 빼다 그런데 그러나 그런데 그치다 그저 그런데 의외로 일단 그저 그게 이상하다 뭐라다 그나마 솔직하다 엄청나다 그저 피순대 그게 그게 그저 음 뭐라다 이상하다 그러나 엄청나다 그러나 아무리 부담스럽다 솔직하다 그리다 그러나 딱하다 그게 나름 부담스럽다 그만큼 심하다 엄청나다 낮다 그만큼 불만족 부담스럽다 고급스럽다 중요하다
price(340개)
가격 비싸다 대비 착하다 대가 비싸다 대도 착하다 대비 대가 대도 사악 그래도 하지만 그렇다 그래서 기대하다 양은 되게